In [1]:
datadownloadpath = './datadownload/Task03_Liver/'
datasavepath = './datapreprocessed/Task03_Liver/'

In [2]:
import glob
import SimpleITK as sitk
import numpy as np
from scipy.ndimage import label, binary_fill_holes
import os

In [4]:
os.mkdir(datasavepath)
os.mkdir(datasavepath + 'Image')
os.mkdir(datasavepath + 'Label')
os.mkdir(datasavepath + 'Labeltumor')
os.mkdir(datasavepath + 'processed')
os.mkdir(datasavepath + 'mask')

In [5]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    resample.SetUseNearestNeighborExtrapolator(True)
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    resample.SetSize( [int(sz+1) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj

def remove_all_but_the_largest_connected_component(image: np.ndarray, for_which_classes: list, volume_per_voxel: float,
                                                   minimum_valid_object_size: dict = None):
    """
    removes all but the largest connected component, individually for each class
    :param image:
    :param for_which_classes: can be None. Should be list of int. Can also be something like [(1, 2), 2, 4].
    Here (1, 2) will be treated as a joint region, not individual classes (example LiTS here we can use (1, 2)
    to use all foreground classes together)
    :param minimum_valid_object_size: Only objects larger than minimum_valid_object_size will be removed. Keys in
    minimum_valid_object_size must match entries in for_which_classes
    :return:
    """
    if for_which_classes is None:
        for_which_classes = np.unique(image)
        for_which_classes = for_which_classes[for_which_classes > 0]

    assert 0 not in for_which_classes, "cannot remove background"
    largest_removed = {}
    kept_size = {}
    for c in for_which_classes:
        if isinstance(c, (list, tuple)):
            c = tuple(c)  # otherwise it cant be used as key in the dict
            mask = np.zeros_like(image, dtype=bool)
            for cl in c:
                mask[image == cl] = True
        else:
            mask = image == c
        # get labelmap and number of objects
        lmap, num_objects = label(mask.astype(int))

        # collect object sizes
        object_sizes = {}
        for object_id in range(1, num_objects + 1):
            object_sizes[object_id] = (lmap == object_id).sum() * volume_per_voxel

        largest_removed[c] = None
        kept_size[c] = None

        if num_objects > 0:
            # we always keep the largest object. We could also consider removing the largest object if it is smaller
            # than minimum_valid_object_size in the future but we don't do that now.
            maximum_size = max(object_sizes.values())
            kept_size[c] = maximum_size

            for object_id in range(1, num_objects + 1):
                # we only remove objects that are not the largest
                if object_sizes[object_id] != maximum_size:
                    # we only remove objects that are smaller than minimum_valid_object_size
                    remove = True
                    if minimum_valid_object_size is not None:
                        remove = object_sizes[object_id] < minimum_valid_object_size[c]
                    if remove:
                        image[(lmap == object_id) & mask] = 0
                        if largest_removed[c] is None:
                            largest_removed[c] = object_sizes[object_id]
                        else:
                            largest_removed[c] = max(largest_removed[c], object_sizes[object_id])
    return image, largest_removed, kept_size

In [6]:
IMG_FOLDER = datadownloadpath + 'imagesTr/'
SEG_FOLDER = datadownloadpath + 'labelsTr/'
imgs = glob.glob(IMG_FOLDER + "/liver_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER + "/liver_*.nii.gz")) ]

pids = [pid.split("_")[-1].split(".")[0] for pid in imgs]

In [7]:
imgs

['./datadownload/Task03_Liver/imagesTr/liver_0.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_1.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_10.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_100.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_101.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_102.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_103.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_104.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_105.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_106.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_107.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_108.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_109.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_11.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_110.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_111.nii.gz',
 './datadownload/Task03_Liver/imagesTr/liver_112.nii.gz',
 './datadownload/Tas

In [8]:
segs

['./datadownload/Task03_Liver/labelsTr/liver_0.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_1.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_10.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_100.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_101.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_102.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_103.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_104.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_105.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_106.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_107.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_108.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_109.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_11.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_110.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_111.nii.gz',
 './datadownload/Task03_Liver/labelsTr/liver_112.nii.gz',
 './datadownload/Tas

In [9]:
pids

['0',
 '1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99']

## Step 1, Data resample

In [10]:
targetspacing = [1.9, 1.9, 2.47]

In [11]:
for (imgfile, lblfile, pid) in zip(imgs, segs, pids):
    savename = 'liver_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    seg_obj = sitk.ReadImage( lblfile )
    img_spa_ori = img_obj.GetSpacing()
    res_img_o = resample_by_res(img_obj, targetspacing, interpolator = sitk.sitkLinear,
                                    logging = True)
    res_lb_o = resample_lb_by_res(seg_obj, targetspacing, interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)
    sitk.WriteImage(res_img_o, datasavepath + 'Image/' + savename, True) 
    sitk.WriteImage(res_lb_o, datasavepath + 'Label/' + savename, True) 

Spacing: (0.703125, 0.703125, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 75) -> [189.47368421 189.47368421 151.82186235]
Label values: [0 1 2]
Spacing: (0.703125, 0.703125, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 75) -> [189.47368421 189.47368421 151.82186235]
Spacing: (0.703125, 0.703125, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 75) -> [189.47368421 189.47368421 151.82186235]
Spacing: (0.703125, 0.703125, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 75) -> [189.47368421 189.47368421 151.82186235]
Spacing: (0.6757810115814209, 0.6757810115814209, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 123) -> [182.10519891 182.10519891 248.98785425]
Label values: [0 1 2]
Spacing: (0.6757810115814209, 0.6757810115814209, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 123) -> [182.10519891 182.10519891 248.98785425]
Spacing: (0.6757810115814209, 0.6757810115814209, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 123) -> [182.10519891 182.10519891 248.98785425]
Spacing: (0.6757810115814209, 0.6757810115814209, 5.0) -> [1

Spacing: (0.7200000286102295, 0.7200000286102295, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 816) -> [194.02106034 194.02106034 264.29150191]
Spacing: (0.7200000286102295, 0.7200000286102295, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 816) -> [194.02106034 194.02106034 264.29150191]
Spacing: (0.7820000052452087, 0.7820000052452087, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 761) -> [210.72842247 210.72842247 246.47773647]
Label values: [0 1 2]
Spacing: (0.7820000052452087, 0.7820000052452087, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 761) -> [210.72842247 210.72842247 246.47773647]
Spacing: (0.7820000052452087, 0.7820000052452087, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 761) -> [210.72842247 210.72842247 246.47773647]
Spacing: (0.7820000052452087, 0.7820000052452087, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 761) -> [210.72842247 210.72842247 246.47773647]
Spacing: (0.6399999856948853, 0.6399999856948853, 0.80

Spacing: (0.6992188096046448, 0.6992188096046448, 1.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 407) -> [188.42106869 188.42106869 247.1659919 ]
Spacing: (0.6992188096046448, 0.6992188096046448, 1.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 407) -> [188.42106869 188.42106869 247.1659919 ]
Spacing: (0.712890625, 0.712890625, 1.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 410) -> [192.10526316 192.10526316 248.98785425]
Label values: [0 1 2]
Spacing: (0.712890625, 0.712890625, 1.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 410) -> [192.10526316 192.10526316 248.98785425]
Spacing: (0.712890625, 0.712890625, 1.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 410) -> [192.10526316 192.10526316 248.98785425]
Spacing: (0.712890625, 0.712890625, 1.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 410) -> [192.10526316 192.10526316 248.98785425]
Spacing: (0.6640625, 0.6640625, 1.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 401) -> [178.94736842 178.94736842 243.52226721]
Label values: [0 1 2]
Spacing: (0.6640625, 0.6640625, 1.5) -> [1.9, 1.9,

Label values: [0 1 2]
Spacing: (0.802734375, 0.802734375, 0.7999879717826843) -> [1.9, 1.9, 2.47]
Size (512, 512, 574) -> [216.31578947 216.31578947 185.90813595]
Spacing: (0.802734375, 0.802734375, 0.7999879717826843) -> [1.9, 1.9, 2.47]
Size (512, 512, 574) -> [216.31578947 216.31578947 185.90813595]
Spacing: (0.802734375, 0.802734375, 0.7999879717826843) -> [1.9, 1.9, 2.47]
Size (512, 512, 574) -> [216.31578947 216.31578947 185.90813595]
Spacing: (0.767578125, 0.767578125, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 437) -> [206.84210526 206.84210526 176.92307692]
Label values: [0 1 2]
Spacing: (0.767578125, 0.767578125, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 437) -> [206.84210526 206.84210526 176.92307692]
Spacing: (0.767578125, 0.767578125, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 437) -> [206.84210526 206.84210526 176.92307692]
Spacing: (0.767578125, 0.767578125, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 437) -> [206.84210526 206.84210526 176.92307692]
Spacing: (0.9765620231628418,

Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 124) -> [269.47368421 269.47368421  50.20242915]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 124) -> [269.47368421 269.47368421  50.20242915]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 111) -> [269.47368421 269.47368421  44.93927126]
Label values: [0 1 2]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 111) -> [269.47368421 269.47368421  44.93927126]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 111) -> [269.47368421 269.47368421  44.93927126]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 111) -> [269.47368421 269.47368421  44.93927126]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 122) -> [269.47368421 269.47368421  49.39271255]
Label values: [0 1 2]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 122) -> [269.47368421 269.47368421  49.39271255]
Spacing: (1.0, 1.0, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 122) -> [269.47

Spacing: (0.90234375, 0.90234375, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 227) -> [243.15789474 243.15789474 275.70850202]
Label values: [0 1 2]
Spacing: (0.90234375, 0.90234375, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 227) -> [243.15789474 243.15789474 275.70850202]
Spacing: (0.90234375, 0.90234375, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 227) -> [243.15789474 243.15789474 275.70850202]
Spacing: (0.90234375, 0.90234375, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 227) -> [243.15789474 243.15789474 275.70850202]
Spacing: (0.8359379768371582, 0.8359379768371582, 2.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 237) -> [225.26328639 225.26328639 239.87854251]
Label values: [0 1 2]
Spacing: (0.8359379768371582, 0.8359379768371582, 2.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 237) -> [225.26328639 225.26328639 239.87854251]
Spacing: (0.8359379768371582, 0.8359379768371582, 2.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 237) -> [225.26328639 225.26328639 239.87854251]
Spacing: (0.8359379768371582, 0.83593797

Spacing: (0.7773439884185791, 0.7773439884185791, 2.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 230) -> [209.47374846 209.47374846 186.23481781]
Spacing: (0.625, 0.625, 0.7989500164985657) -> [1.9, 1.9, 2.47]
Size (512, 512, 513) -> [168.42105263 168.42105263 165.93577266]
Label values: [0 1 2]
Spacing: (0.625, 0.625, 0.7989500164985657) -> [1.9, 1.9, 2.47]
Size (512, 512, 513) -> [168.42105263 168.42105263 165.93577266]
Spacing: (0.625, 0.625, 0.7989500164985657) -> [1.9, 1.9, 2.47]
Size (512, 512, 513) -> [168.42105263 168.42105263 165.93577266]
Spacing: (0.625, 0.625, 0.7989500164985657) -> [1.9, 1.9, 2.47]
Size (512, 512, 513) -> [168.42105263 168.42105263 165.93577266]
Spacing: (0.7128909826278687, 0.7128909826278687, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 86) -> [192.10535953 192.10535953 174.08906883]
Label values: [0 1 2]
Spacing: (0.7128909826278687, 0.7128909826278687, 5.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 86) -> [192.10535953 192.10535953 174.08906883]
Spacing: (0.7128909826

Spacing: (0.7851560115814209, 0.7851560115814209, 2.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 198) -> [211.57888312 211.57888312 200.4048583 ]
Spacing: (0.7851560115814209, 0.7851560115814209, 2.5) -> [1.9, 1.9, 2.47]
Size (512, 512, 198) -> [211.57888312 211.57888312 200.4048583 ]
Spacing: (0.796875, 0.796875, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 147) -> [214.73684211 214.73684211 178.54251012]
Label values: [0 1 2]
Spacing: (0.796875, 0.796875, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 147) -> [214.73684211 214.73684211 178.54251012]
Spacing: (0.796875, 0.796875, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 147) -> [214.73684211 214.73684211 178.54251012]
Spacing: (0.796875, 0.796875, 3.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 147) -> [214.73684211 214.73684211 178.54251012]
Spacing: (0.84765625, 0.84765625, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 512, 541) -> [228.42105263 228.42105263 219.02834008]
Label values: [0 1 2]
Spacing: (0.84765625, 0.84765625, 1.0) -> [1.9, 1.9, 2.47]
Size (512, 51

Spacing: (0.8579999804496765, 0.8579999804496765, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 751) -> [231.20841578 231.20841578 243.23887002]
Spacing: (0.6710000038146973, 0.6710000038146973, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 836) -> [180.81684313 180.81684313 270.7692348 ]
Label values: [0 1 2]
Spacing: (0.6710000038146973, 0.6710000038146973, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 836) -> [180.81684313 180.81684313 270.7692348 ]
Spacing: (0.6710000038146973, 0.6710000038146973, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 836) -> [180.81684313 180.81684313 270.7692348 ]
Spacing: (0.6710000038146973, 0.6710000038146973, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 836) -> [180.81684313 180.81684313 270.7692348 ]
Spacing: (0.6620000004768372, 0.6620000004768372, 0.800000011920929) -> [1.9, 1.9, 2.47]
Size (512, 512, 696) -> [178.39157908 178.39157908 225.42510457]
Label values: [0 1 2]
Spacing: (0.6620000004768372, 0.

## Step 2, Intensity Normalization

In [12]:
IMG_FOLDER_SAVE = datasavepath + 'Image/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/liver_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [13]:
pixel_mean = 98.0451
pixel_std = 32.4977
intensity_05percent = -8.6201
intensity_995percent = 182.1394

In [14]:
for (imgfile, pid) in zip(imgs, pids):
    savename = 'liver_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    
    array = sitk.GetArrayFromImage(img_obj)

    array[array < intensity_05percent] = intensity_05percent
    array[array > intensity_995percent] = intensity_995percent
    array = (array - pixel_mean) / pixel_std

    normalized_img = sitk.GetImageFromArray(array)
    normalized_img.CopyInformation(img_obj)
    
    sitk.WriteImage(normalized_img, datasavepath + 'processed/' + savename, True) 
    print('Id ' + pid + ' normalized')

Id 0 normalized
Id 1 normalized
Id 10 normalized
Id 100 normalized
Id 101 normalized
Id 102 normalized
Id 103 normalized
Id 104 normalized
Id 105 normalized
Id 106 normalized
Id 107 normalized
Id 108 normalized
Id 109 normalized
Id 11 normalized
Id 110 normalized
Id 111 normalized
Id 112 normalized
Id 113 normalized
Id 114 normalized
Id 115 normalized
Id 116 normalized
Id 117 normalized
Id 118 normalized
Id 119 normalized
Id 12 normalized
Id 120 normalized
Id 121 normalized
Id 122 normalized
Id 123 normalized
Id 124 normalized
Id 125 normalized
Id 126 normalized
Id 127 normalized
Id 128 normalized
Id 129 normalized
Id 13 normalized
Id 130 normalized
Id 14 normalized
Id 15 normalized
Id 16 normalized
Id 17 normalized
Id 18 normalized
Id 19 normalized
Id 2 normalized
Id 20 normalized
Id 21 normalized
Id 22 normalized
Id 23 normalized
Id 24 normalized
Id 25 normalized
Id 26 normalized
Id 27 normalized
Id 28 normalized
Id 29 normalized
Id 3 normalized
Id 30 normalized
Id 31 normalized
Id 3

## Step 3, Generate Binary Segmentation Masks

In [15]:
SEG_FOLDER_SAVE = datasavepath + 'Label/'
segs = [ fid for fid in sorted(glob.glob(SEG_FOLDER_SAVE + "/liver_*.nii.gz")) ]

In [16]:
for (segfile, pid) in zip(segs, pids):
    savename = 'liver_' + pid + '.nii.gz'
    seg_obj = sitk.ReadImage( segfile )
    
    array = sitk.GetArrayFromImage(seg_obj)

    array[array == 1] = 0
    array[array == 2] = 1

    binary_seg = sitk.GetImageFromArray(array)
    binary_seg.CopyInformation(seg_obj)
    
    sitk.WriteImage(binary_seg, datasavepath + 'Labeltumor/' + savename, True) 
    print('Id ' + pid + ' generated')

Id 0 generated
Id 1 generated
Id 10 generated
Id 100 generated
Id 101 generated
Id 102 generated
Id 103 generated
Id 104 generated
Id 105 generated
Id 106 generated
Id 107 generated
Id 108 generated
Id 109 generated
Id 11 generated
Id 110 generated
Id 111 generated
Id 112 generated
Id 113 generated
Id 114 generated
Id 115 generated
Id 116 generated
Id 117 generated
Id 118 generated
Id 119 generated
Id 12 generated
Id 120 generated
Id 121 generated
Id 122 generated
Id 123 generated
Id 124 generated
Id 125 generated
Id 126 generated
Id 127 generated
Id 128 generated
Id 129 generated
Id 13 generated
Id 130 generated
Id 14 generated
Id 15 generated
Id 16 generated
Id 17 generated
Id 18 generated
Id 19 generated
Id 2 generated
Id 20 generated
Id 21 generated
Id 22 generated
Id 23 generated
Id 24 generated
Id 25 generated
Id 26 generated
Id 27 generated
Id 28 generated
Id 29 generated
Id 3 generated
Id 30 generated
Id 31 generated
Id 32 generated
Id 33 generated
Id 34 generated
Id 35 generat

## Step 4, Generate Body Masks

In [17]:
IMG_FOLDER_SAVE = datasavepath + 'Image/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/liver_*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [18]:
threshold = -200

In [19]:
for (imgfile, pid) in zip(imgs, pids):
    savename = 'liver_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    array = sitk.GetArrayFromImage(img_obj)
    array = array > threshold

    array_component = remove_all_but_the_largest_connected_component(array, [1], targetspacing)

    array_mask = array_component[0].astype(int)
    array_mask = binary_fill_holes(array_mask).astype(float)

    mask_img = sitk.GetImageFromArray(array_mask)
    mask_img.CopyInformation(img_obj)
    
    sitk.WriteImage(mask_img, datasavepath + 'mask/' + savename, True) 
    print('Id ' + pid + ' mask generated')

Id 0 mask generated
Id 1 mask generated
Id 10 mask generated
Id 100 mask generated
Id 101 mask generated
Id 102 mask generated
Id 103 mask generated
Id 104 mask generated
Id 105 mask generated
Id 106 mask generated
Id 107 mask generated
Id 108 mask generated
Id 109 mask generated
Id 11 mask generated
Id 110 mask generated
Id 111 mask generated
Id 112 mask generated
Id 113 mask generated
Id 114 mask generated
Id 115 mask generated
Id 116 mask generated
Id 117 mask generated
Id 118 mask generated
Id 119 mask generated
Id 12 mask generated
Id 120 mask generated
Id 121 mask generated
Id 122 mask generated
Id 123 mask generated
Id 124 mask generated
Id 125 mask generated
Id 126 mask generated
Id 127 mask generated
Id 128 mask generated
Id 129 mask generated
Id 13 mask generated
Id 130 mask generated
Id 14 mask generated
Id 15 mask generated
Id 16 mask generated
Id 17 mask generated
Id 18 mask generated
Id 19 mask generated
Id 2 mask generated
Id 20 mask generated
Id 21 mask generated
Id 22 